In [1]:
from src import environments
import nest_asyncio
nest_asyncio.apply()

import tensorflow as tf
from gym_derk.envs import DerkEnv
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np
from src.environments import EnvPerso, EnvTensorOne
import tf_agents
import reverb
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.specs import tensor_spec
from tf_agents.policies import py_tf_eager_policy
import reverb
from tf_agents.drivers import py_driver

from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
import tensorflow as tf
from tf_agents.agents.ppo import ppo_agent
from tf_agents.networks import actor_distribution_network, value_network

2023-07-06 21:30:31.145326: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
num_iterations = 20000 

initial_collect_steps = 100 
collect_steps_per_iteration =   1
replay_buffer_max_length = 1000

batch_size = 64  
learning_rate = 1e-3  
log_interval = 200 

num_eval_episodes = 10 
eval_interval = 1000  

In [3]:
env = EnvTensorOne()

In [4]:
# Define the network for the agent.
actor_net = actor_distribution_network.ActorDistributionNetwork(
    env.observation_tensor_spec(),
    env.action_tensor_spec())

value_net = value_network.ValueNetwork(
    env.observation_tensor_spec())

# Define the keras optimizer.
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Define the train step counter.
train_step_counter = tf.Variable(0)

# Create the PPO agent.
agent = ppo_agent.PPOAgent(
    env.time_step_spec(),
    env.action_tensor_spec(),
    optimizer,
    actor_net=actor_net,
    value_net=value_net,
    train_step_counter=train_step_counter)

# Initialize the agent.
agent.initialize()

eval_policy = agent.policy
collect_policy = agent.collect_policy

In [5]:
collect_policy

In [6]:
random_policy = random_tf_policy.RandomTFPolicy(env.time_step_spec(),
                                                env.action_spec())

In [7]:
class DebugReverbAddTrajectoryObserver(reverb_utils.ReverbAddTrajectoryObserver):
    def __call__(self, trajectory):
        print(trajectory)
        super().__call__(trajectory)

In [8]:
table_name = 'test_table'
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)

table = reverb.Table(
    table_name,
    max_size=replay_buffer_max_length,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    agent.collect_data_spec,
    table_name=table_name,
    sequence_length=2,
    local_server=reverb_server)

rb_observer = DebugReverbAddTrajectoryObserver(
  replay_buffer.py_client,
  table_name,
  sequence_length=2)

[reverb/cc/platform/tfrecord_checkpointer.cc:162]  Initializing TFRecordCheckpointer in /tmp/tmpz_mfac_r.
[reverb/cc/platform/tfrecord_checkpointer.cc:567] Loading latest checkpoint from /tmp/tmpz_mfac_r
[reverb/cc/platform/default/server.cc:71] Started replay server on port 46599


In [9]:
agent.collect_data_spec

_TupleWrapper(Trajectory(
{'action': BoundedTensorSpec(shape=(15,), dtype=tf.float32, name='action', minimum=array(-1., dtype=float32), maximum=array(1., dtype=float32)),
 'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'next_step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type'),
 'observation': BoundedTensorSpec(shape=(64,), dtype=tf.float32, name='observation', minimum=array(-1., dtype=float32), maximum=array(1., dtype=float32)),
 'policy_info': DictWrapper({'dist_params': DictWrapper({'loc': TensorSpec(shape=(15,), dtype=tf.float32, name='NormalProjectionNetwork_loc'), 'scale': TensorSpec(shape=(15,), dtype=tf.float32, name='NormalProjectionNetwork_scale')})}),
 'reward': BoundedTensorSpec(shape=(), dtype=tf.float32, name='reward', minimum=array(0., dtype=float32), maximum=array(inf, dtype=float32)),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')}))

In [10]:
py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      random_policy, use_tf_function=True),
    [rb_observer],
    max_steps=initial_collect_steps).run(env.reset())

<class 'numpy.ndarray'>
Trajectory(
{'action': array([ 0.21042061, -0.30629253,  0.01914287,  0.24728942,  0.33934212,
        0.2127502 ,  0.        ,  0.        ,  0.5569837 ,  0.75553226,
        0.8224182 ,  0.8092327 ,  0.1810453 ,  0.        ,  0.        ],
      dtype=float32),
 'discount': array(1., dtype=float32),
 'next_step_type': array(1, dtype=int32),
 'observation': array([ 1.        ,  1.        ,  0.24078695, -0.8728948 ,  0.17336386,
       -0.16567287,  0.2675242 , -0.39621603,  1.        , -0.02919272,
        0.79562455, -0.09462335,  0.6693634 , -0.03331383,  0.6873951 ,
        0.01854582,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
        0.11146335,  0.11146335,  0.62728584, -0.03068253, -0.13041624,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.     

(TimeStep(
 {'discount': array(1., dtype=float32),
  'observation': array([ 1.        ,  1.        ,  0.5111935 , -0.7707966 ,  0.15485238,
         0.9167367 ,  0.06586765,  0.720049  ,  0.71806055,  0.2648214 ,
         0.1638141 ,  0.372696  ,  0.11164905, -0.8985493 ,  0.164289  ,
         0.67954135,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.01105312, -0.03438446,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.       

In [11]:
iter(replay_buffer.as_dataset()).next()

InvalidArgumentError: {{function_node __wrapped__MakeIterator_device_/job:localhost/replica:0/task:0/device:CPU:0}} Requested incompatible tensor at flattened index 0 from table 'test_table'.  Requested (dtype, shape): (int32, [1]).  Signature (dtype, shape): (int32, []).  Table signature: 0: Tensor<name: 'step_type/step_type', dtype: int32, shape: []>, 1: Tensor<name: 'observation/observation', dtype: float, shape: [64]>, 2: Tensor<name: 'action/action', dtype: float, shape: [15]>, 3: Tensor<name: 'policy_info/dist_params/loc/NormalProjectionNetwork_loc', dtype: float, shape: [15]>, 4: Tensor<name: 'policy_info/dist_params/scale/NormalProjectionNetwork_scale', dtype: float, shape: [15]>, 5: Tensor<name: 'next_step_type/step_type', dtype: int32, shape: []>, 6: Tensor<name: 'reward/reward', dtype: float, shape: []>, 7: Tensor<name: 'discount/discount', dtype: float, shape: []> [Op:MakeIterator]